In [1]:
import numpy as np
import pandas as pd
import argparse
from sklearn.tree import DecisionTreeClassifier, plot_tree, _tree
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import pydotplus
from matplotlib import pyplot as plt
from labels import mac_to_label
import json
import math

In [2]:
Set1 = pd.read_csv('data.csv').values.tolist()
X = [i[0:3] for i in Set1]
Y =[i[3] for i in Set1]

In [3]:
predict_Yt = []
with open('compressed_tree.json', 'r') as file:
    data = json.load(file)
    classes = data["classes"]
    for x in X:
        counter = 0
        class_set = []
        for feature in reversed(data['layers']): #Have to reverse this list due to structure of the data.csv file and how it aligns with the compressed tree layers
            for node in data['layers'][feature]:
                if node['min'] is None:
                    if x[counter] < node['max']:
                        class_set.append(node['classes'])
                        break #is this an issue?
                    else:
                        continue
                elif node['max'] is None:
                    if node['min'] < x[counter]:
                        class_set.append(node['classes'])
                        break #is this an issue?
                    else:
                        continue
                elif node['min'] < x[counter] and x[counter] < node['max']:
                    class_set.append(node['classes'])
                    break #is this an issue?

            counter += 1
        result = set(class_set[0])
        for s in class_set[1:]:
            result.intersection_update(s)

        #predict_Yt.append(list(result))
        #print(result)
        if len(result) == 1:
            prediction = list(result)[0]
            pred_class = classes[prediction]
            predict_Yt.append(pred_class)
        else:
            predict_Yt.append(None)

In [4]:
correct = 0
for i in range(len(Y)):
    prediction = predict_Yt[i]
    if prediction != None and Y[i] == prediction:
        correct += 1

print(correct / len(Y))

0.8448217242194891
